<a href="https://colab.research.google.com/github/SD0313/Skin-Disease/blob/master/VGG_Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
root_path = 'gdrive/My Drive/SkinDisease_Data'

In [0]:
import os
import numpy as np
import pandas as pd

IMAGE_DIR = os.path.join(root_path, 'ISIC_2019_Training_Input')
panda_path = os.path.join(root_path, 'ISIC_2019_Training_GroundTruth.csv')

In [4]:
print(f'This is the image dir: {IMAGE_DIR}')
print(f'This is the csv filepath: {panda_path}')

This is the image dir: gdrive/My Drive/SkinDisease_Data/ISIC_2019_Training_Input
This is the csv filepath: gdrive/My Drive/SkinDisease_Data/ISIC_2019_Training_GroundTruth.csv


In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.vgg16 import preprocess_input, VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

In [0]:
def preprocess(df):
  for index, img in enumerate(df.image):
    img = img+'.jpg'
    df.image[index]=img
  df.drop(['UNK'], axis=1, inplace=True)

In [0]:
def train_val_test_split(df, test_len=1000, val_ratio=0.2):
  test_rows = (np.random.rand(1000)*df.shape[0]).astype(int)
  test_df =  df.iloc[test_rows]
  test_df = test_df.reset_index().drop(['index'], axis=1)
  df.drop(test_rows, axis=0, inplace=True)
  df = df.reset_index().drop(['index'], axis=1)
  val_rows = (np.random.rand(int(val_ratio*df.shape[0]))*df.shape[0]).astype(int)
  val_df = df.iloc[val_rows]
  df.drop(val_rows, axis=0, inplace=True)
  test_df = test_df.reset_index().drop(['index'], axis=1)
  df = df.reset_index().drop(['index'], axis=1)
  return df, val_df, test_df


In [8]:
full_df = pd.read_csv(panda_path)
preprocess(full_df)
train_df, val_df, test_df = train_val_test_split(full_df)
train_df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC
0,ISIC_0000001.jpg,0,1,0,0,0,0,0,0
1,ISIC_0000002.jpg,1,0,0,0,0,0,0,0
2,ISIC_0000004.jpg,1,0,0,0,0,0,0,0
3,ISIC_0000006.jpg,0,1,0,0,0,0,0,0
4,ISIC_0000007.jpg,0,1,0,0,0,0,0,0


In [0]:
def basic_vgg(input_shape=(224, 224, 3), num_classes=8):
  new_input = Input(shape=input_shape)
  model = VGG16(weights=None, input_tensor=new_input, classes=num_classes)
  model.compile(optimizer=Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [21]:
base_model = basic_vgg()
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [0]:
#sequential API
def vgg_model(input_shape=(224, 224, 3)):
  model = Sequential()
  model.add(VGG16(include_top=False, weights='imagenet', input_shape=input_shape))
  
  model.add(GlobalAveragePooling2D())
  #model.add(Flatten())

  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.25))

  model.add(Dense(1024))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.5))

  model.add(Dense(9, activation='sigmoid'))
  model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
  print('Model has compiled')
  return model

In [10]:
vgg16_model = vgg_model(input_shape=(224, 224, 3))

58892288/58889256 [==============================] - 1s 0us/step
Model has compiled


In [11]:
vgg16_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              525312    
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
activation (Activation)      (None, 1024)              0

In [0]:
def get_train_gen(df, img_path=IMAGE_DIR, target_size=(224, 224)):
  data_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                horizontal_flip=True,
                                width_shift_range=0.2,
                                height_shift_range=0.2)
  return data_gen.flow_from_dataframe(dataframe=df, directory=img_path, 
                                      x_col='image', y_col=list(df.columns)[1:],
                                      batch_size=64, shuffle=True, class_mode='raw', 
                                      target_size=target_size)

def get_val_test_gen(val_df, test_df, img_path=IMAGE_DIR, target_size=(224, 224)):
  data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
  val = data_gen.flow_from_dataframe(dataframe=val_df, directory=img_path, 
                                      x_col='image', y_col=list(val_df.columns)[1:],
                                      batch_size=64, shuffle=True, class_mode='raw', 
                                      target_size=target_size)
  test = data_gen.flow_from_dataframe(dataframe=test_df, directory=img_path, 
                                      x_col='image', batch_size=1, shuffle=True, class_mode=None, 
                                      target_size=target_size)
  return val, test

In [13]:
train_generator = get_train_gen(train_df)
valid_generator, test_generator = get_val_test_gen(val_df, test_df)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 2 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 19926 validated image filenames.
Found 4870 validated image filenames.
Found 1000 validated image filenames.


In [0]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
history = base_model.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN, validation_data=valid_generator,
                        validation_steps=STEP_SIZE_VALID, epochs=2)

Epoch 1/2
311/311 [==============================] - ETA: 0s - loss: 1.2854 - accuracy: 0.5343 